<a href="https://colab.research.google.com/github/HausReport/MightyLogic/blob/master/Notebooks/Mighty_Party_Doodles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set up environment


In [197]:

import sys
# need newer plotly/plotly_express than collab provides?
#!{sys.executable} -m pip install --force-reinstall --upgrade "plotly"
#!{sys.executable} -m pip install --force-reinstall --upgrade "plotly_express"

#
# Moving some of the guts to GitHub and refactoring 
#
!{sys.executable} -m pip install --upgrade --force-reinstall "git+https://github.com/HausReport/MightyLogic.git#egg=MightyLogic"

#
# Imports
#
from random import randrange
from scipy.signal import convolve2d
from enum import Enum, auto
from pprint import pprint
import plotly.figure_factory as ff

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import math

#import sklearn
#from sklearn.preprocessing import PowerTransformer, QuantileTransformer

  Cloning https://github.com/HausReport/MightyLogic.git to /tmp/pip-install-wapqqjux/mightylogic_22376526f0a840b68985f16740e65861
  Running command git clone -q https://github.com/HausReport/MightyLogic.git /tmp/pip-install-wapqqjux/mightylogic_22376526f0a840b68985f16740e65861
  Created wheel for MightyLogic: filename=MightyLogic-0.1-py3-none-any.whl size=4601 sha256=8c89cdec4c1e34a736870db4aa84bf346895abb8155bdcb6bc42b90764815df7
  Stored in directory: /tmp/pip-ephem-wheel-cache-vuup9711/wheels/84/7a/55/d74e49b550ce0fddb2379db726853920650df706dedea8daae
Successfully built MightyLogic
  Attempting uninstall: MightyLogic
    Found existing installation: MightyLogic 0.1
    Uninstalling MightyLogic-0.1:
      Successfully uninstalled MightyLogic-0.1


In [207]:

from MightyLogic.Currency import Currency as Currency
from MightyLogic.Values import Values as Values

Values.GOLD

0.6

# Basic Structures & Functions

In [138]:
#
# Types of in-game currencies
#
class Currency(Enum):
  COMMON_SOUL = auto()
  RARE_SOUL = auto()
  SOUL_DUST = auto()
  EPIC_SOUL = auto()
  GOLD = auto()
  KILO_SOUL = auto()
  LEGENDARY_SOUL = auto()
  CONTRIBUTION = auto()
  INFLUENCE = auto()
  SPARK = auto()
  GEM = auto()

#
# Relative currency values, based on https://mightyparty.fandom.com/wiki/Currency
# with some tweaks
#
value_dict = {
    Currency.COMMON_SOUL:0.001 * 1.0,
    Currency.RARE_SOUL:0.01    * 1.0,
    Currency.SOUL_DUST:0.05    * 1.0,
    Currency.EPIC_SOUL:0.1     * 1.0,
    Currency.GOLD:	0.6        * 1.0,
    Currency.KILO_SOUL:1       * 1.0,
    Currency.LEGENDARY_SOUL:2  * 1.0,
    Currency.CONTRIBUTION:15   * 1.0,
    Currency.INFLUENCE:15      * 1.0,
    Currency.SPARK:20          * 1.0,
    Currency.GEM:60            * 1.0}

In [176]:
#
# A general wrapper for rewards consisting of mixtures of currencies.
# Unless a specific non-zero value is given, it's computed on-the-fly.
#
def wrapDict(name="Unknown", location="?:?",value=0,common=0,rare=0, soul_dust=0, epic=0, gold=0, legendary=0, contrib=0, influence=0, spark=0, gem=0):
  val = value
  if val == 0:
    val += value_dict[Currency.COMMON_SOUL] * common
    val += value_dict[Currency.RARE_SOUL] * rare
    val += value_dict[Currency.SOUL_DUST] * soul_dust
    val += value_dict[Currency.EPIC_SOUL] * epic
    val += value_dict[Currency.GOLD] * gold
    val += value_dict[Currency.KILO_SOUL] * 0
    val += value_dict[Currency.LEGENDARY_SOUL] * legendary
    val += value_dict[Currency.CONTRIBUTION] * contrib
    val += value_dict[Currency.INFLUENCE] * influence
    val += value_dict[Currency.SPARK] * spark
    val += value_dict[Currency.GEM] * gem

  return {  "name":name,  #for tiles
            "location":location,  #for tiles
            "value":val,
            Currency.COMMON_SOUL: common,
            Currency.RARE_SOUL: rare,
            Currency.SOUL_DUST: soul_dust,
            Currency.EPIC_SOUL: epic,
            Currency.GOLD: gold,
            Currency.LEGENDARY_SOUL: legendary,
            Currency.CONTRIBUTION: contrib,
            Currency.INFLUENCE: influence,
            Currency.SPARK: spark,
            Currency.GEM: gem
          }

# Event Chest Functions

In [140]:
#
# Various types of event chest.  Values for tier 2, chapter 3
# See https://mightyparty.fandom.com/wiki/Global_Events#Event_Chest_Contents
#
def rare_chest(verbose=False):
  branch = randrange(0,99)
  if branch<75:
    souls = randrange(80,220)
    if verbose:
      print(f"{souls} rare souls")
    return wrapDict(rare=souls)
  elif branch<77:
    souls = 20
    if verbose:
      print(f"{souls} epic souls")
    return wrapDict(epic=souls)
  elif branch<97:
    souls = randrange(175,300)
    if verbose:
      print(f"{souls} sparks")
    return wrapDict(spark=souls)
  else:
    souls = randrange(50,100)
    if verbose:
      print(f"{souls} gems")
    return wrapDict(gem=souls)

def epic_chest(verbose=False):
  branch = randrange(0,99)
  if branch<75:
    souls = randrange(80,500)
    if verbose:
      print(f"{souls} epic souls")
    return wrapDict(epic=souls)
  elif branch<77:
    souls = 20
    if verbose:
      print(f"{souls} legendary souls")
    return wrapDict(legendary=souls)
  elif branch<97:
    souls = randrange(750,2000)
    if verbose:
      print(f"{souls} sparks")
    return wrapDict(spark=souls)
  else:
    souls = randrange(150,300)
    if verbose:
      print(f"{souls} gems")
    return wrapDict(gem=souls)

def legendary_chest(verbose=False):
  branch = randrange(0,99)
  if branch<62:
    souls = randrange(40,160)
    if verbose:
      print(f"{souls} legendary souls")
    return wrapDict(legendary=souls)
  elif branch<97:
    souls = randrange(3500,10000)
    if verbose:
      print(f"{souls} sparks")
    return wrapDict(spark=souls)
  else:
    souls = 1000
    if verbose:
      print(f"{souls} gems")
    return wrapDict(gem=souls)


# Simulate Buying Event Chests

In [141]:
#
# Simulate spending 5000 sparks on various types of event chests
#

def k_sim(spx, cost, verbose=False):
  df = pd.DataFrame()
  tot = 0
  sparks = spx
  rolls = 0
  spent = 0 
  while sparks>(cost-1):
    ret = None
    if cost==250:
      ret = rare_chest()
    elif cost==1000:
      ret = epic_chest()
    elif cost==5000:
      ret = legendary_chest()
    rolls +=1
    tot += ret["value"]
    df = df.append(ret,ignore_index=True)
    sparks = sparks + ret[Currency.SPARK]
    spent = spent+cost
    sparks = sparks - cost


  cost = 5000 * value_dict[Currency.SPARK]
  eff = tot/cost

  if verbose:  
    print(f"Rolls: {rolls}")
    print(f"Total value: {tot}")
    print(f"Total efficiency: {eff}")
    print(f"Spent: {spent}")

  return df, rolls, spent  

def fivek_sim(cost, verbose=False):
  return k_sim(5000, cost, verbose)

def fivek_rare(verbose=False):
  return fivek_sim(250, verbose=verbose)

def fivek_legend(verbose=False):
  return fivek_sim(5000,verbose=verbose)

def fivek_epic(verbose=False):
  return fivek_sim(1000, verbose=verbose)

In [142]:
#
#Total value gained spending 5000 sparks on epic chests
#
df, rolls, spent = fivek_epic(verbose=True)
df['value'].sum()

Rolls: 7
Total value: 42693.6
Total efficiency: 0.426936
Spent: 7000


42693.6

In [143]:
#
# Simulate spending 5000 sparks on epic chests 1000 times and show resulting value obtained
#
results = []
for i in range(0,1000):
  df, rolls, spent = fivek_epic()
  results.append(df['value'].sum())

#pprint(results)
fig = px.histogram(pd.DataFrame(results,columns=["Value"]),x="Value")
fig.show()

In [144]:
#
# Simulate spending 5000 sparks on epic chests 1000 times and show resulting number of rolls possible
#
results = []
for i in range(0,1000):
  df, rolls, spent = fivek_epic()
  results.append(rolls)

#pprint(results)
df = pd.DataFrame(results,columns=["Rolls"])
fig = px.histogram(df,x="Rolls")
fig.show()

In [145]:
#
# Above, but tabular format
#
import numpy as np

df.groupby(['Rolls'], as_index=False).size()
#.aggregate(np.sum)

,Rolls,size
0,5,408
1,6,285
2,7,117
3,8,73
4,9,44
5,10,30
6,11,23
7,12,8
8,13,8
9,14,2


In [146]:
#
# Simulate spending 5000 sparks on legendary chests 1000 times and show resulting number of rolls possible
#
results = []
for i in range(0,1000):
  df, rolls, spent = fivek_legend()
  results.append(rolls)

#pprint(results)
df = pd.DataFrame(results,columns=["Rolls"])
fig = px.histogram(df,x="Rolls", log_y=True)
fig.update_layout(title='# Rolls From Spending 5000 sparks on Legendary Chests', autosize=False,
                  width=600, height=500,
                  margin=dict(l=65, r=50, b=65, t=90))
fig.show()

In [147]:
#
# Above, but tabular format
#
import numpy as np

df.groupby(['Rolls'], as_index=False).size()
#.aggregate(np.sum)

,Rolls,size
0,1,732
1,2,169
2,3,38
3,4,30
4,5,14
5,6,6
6,7,3
7,8,3
8,9,1
9,11,1


In [148]:
#
# Simulate spending 5000 sparks on epic chests 1000 times and show resulting number of rolls possible
#
results = []
for i in range(0,1000):
  df, rolls, spent = fivek_epic()
  results.append(rolls)

#pprint(results)
df = pd.DataFrame(results,columns=["Rolls"])
fig = px.histogram(df,x="Rolls") #, log_y=True)
fig.update_layout(title='# Rolls From Spending 5000 sparks on Epic Chests', autosize=False,
                  width=600, height=500,
                  margin=dict(l=65, r=50, b=65, t=90))
fig.show()

In [149]:
#
# Simulate spending 5000 sparks on rare chests 1000 times and show resulting number of rolls possible
#
results = []
for i in range(0,1000):
  df, rolls, spent = fivek_rare()
  results.append(rolls)

#pprint(results)
df = pd.DataFrame(results,columns=["Rolls"])
fig = px.histogram(df,x="Rolls") #, log_y=True)
fig.update_layout(title='# Rolls From Spending 5000 sparks on Rare Chests', autosize=False,
                  width=600, height=500,
                  margin=dict(l=65, r=50, b=65, t=90))
fig.show()

In [150]:
fivek_rare()

(       name    value  ...  Currency.SPARK  Currency.GEM
 0   Unknown     1.89  ...             0.0           0.0
 1   Unknown     1.84  ...             0.0           0.0
 2   Unknown  3740.00  ...           187.0           0.0
 3   Unknown  5320.00  ...           266.0           0.0
 4   Unknown  5440.00  ...           272.0           0.0
 5   Unknown     1.83  ...             0.0           0.0
 6   Unknown     1.52  ...             0.0           0.0
 7   Unknown  3780.00  ...           189.0           0.0
 8   Unknown     1.41  ...             0.0           0.0
 9   Unknown     0.98  ...             0.0           0.0
 10  Unknown     2.00  ...             0.0           0.0
 11  Unknown  5520.00  ...           276.0           0.0
 12  Unknown  5080.00  ...           254.0           0.0
 13  Unknown     1.67  ...             0.0           0.0
 14  Unknown     1.05  ...             0.0           0.0
 15  Unknown     1.81  ...             0.0           0.0
 16  Unknown     1.07  ...     

In [151]:
fivek_legend()

(      name  value  ...  Currency.SPARK  Currency.GEM
 0  Unknown  168.0  ...             0.0           0.0
 
 [1 rows x 12 columns], 1, 5000)

In [152]:
#
# Simulate spending 1000 sparks on epic chests 1000 times and show resulting value obtained
#
results = []
for i in range(0,1000):
  df, rolls, spent = k_sim(1000,1000)
  results.append(df['value'].sum())

#pprint(results)
fig = px.histogram(pd.DataFrame(results,columns=["Value"]),x="Value")
fig.show()

In [153]:
#
# Simulate spending 1000 sparks on epic chests 1000 times and show resulting value obtained
#
results = []
for i in range(0,1000):
  df, rolls, spent = k_sim(1000,250)
  results.append(df['value'].sum())

#pprint(results)
fig = px.histogram(pd.DataFrame(results,columns=["Value"]),x="Value")
fig.show()

# Turf War Doodles

In [175]:
#
# Convenience Functions for Turf War tile types
#

# Buildings, alphabetically

def altar(epic, influence, contrib, location="?:?"):
  return wrapDict(name="Altar", location=location, epic=epic,influence=influence, contrib=contrib)

def factory(soul_dust, influence, contrib, location="?:?"):
  return wrapDict(name="Factory", location=location, soul_dust=soul_dust,influence=influence, contrib=contrib)

def gold_mine(gold, influence, contrib, location="?:?"):
  return wrapDict(name="Gold Mine", location=location, gold=gold,influence=influence, contrib=contrib)

def hells_gate(epic, influence, contrib, location="?:?"):
  return wrapDict(name="Hell's Gate", location=location, epic=epic,influence=influence, contrib=contrib)

def lighthouse(rare, influence, contrib, location="?:?"):
  return wrapDict(name="Lighthouse", location=location, rare=rare,influence=influence, contrib=contrib)

def tree_of_life(rare, influence, contrib, location="?:?"):
  return wrapDict(name="Tree of Life", location=location, rare=rare,influence=influence, contrib=contrib)

def witch_hut(rare, influence, contrib, location="?:?"):
  return wrapDict(name="Witch Hut", location=location, rare=rare,influence=influence, contrib=contrib)

# 
# Non-buildings
#
def forest(common, influence, contrib, location="?:?"):
  return wrapDict(name="Forest", location=location, common=common,influence=influence, contrib=contrib)

def mountains(common, contrib, location="?:?"):
  return wrapDict(name="Mountains", location=location, common=common, contrib=contrib)

def plains(common, contrib, location="?:?"):
  return wrapDict(name="Plains", location=location, common=common, contrib=contrib)

In [180]:
#
# A Turf War Map
#

# Use convenience functions to build value array

a1 = forest(1600,0,1000,location="A:1")
a2 = forest(1600,0,1000,location="A:2")
a3 = plains(2600, 0,location="A:3")
a4 = mountains(1600,1000,location="A:4")
a5 = forest(1600,0,1000,location="A:5")
a6 = plains(1600, 0,location="A:6")

b1 = gold_mine(45000,250,2500,location="B:1")
b2 = mountains(2500,2500,location="B:2")
b3 = tree_of_life(1700,300,3500,location="B:3")
b4 = plains(2500,2500,location="B:4")
b5 = mountains(2500,2500,location="B:5")
b6 = forest(1600,0,1500,location="B:6")

c1 = forest(1600,0,1000,location="C:1")
c2 = plains(2500,2500,location="C:2")
c3 = mountains(2500,2500,location="C:3")
c4 = forest(2500,0,2500,location="C:4")
c5 = hells_gate(300,400,4250,location="C:5")
c6 = mountains(1600,1000,location="C:6")

d1 = lighthouse(1000,200,2500,location="D:1")
d2 = forest(2500,0, 2500,location="D:2")
d3 = plains(2500,2500,location="D:3")
d4 = altar(300,400,4250,location="D:4")
d5 = forest(2500,0, 2500,location="D:5")
d6 = plains(2600, 0,location="D:6")

e1 = plains(2600, 0,location="E:1")
e2 = mountains(1600,1000,location="E:2")
e3 = forest(1600,0,1000,location="E:3")
e4 = plains(2600, 0,location="E:4")
e5 = factory(900,250,2500,location="E:5")
e6 = forest(1600,0,1000,location="E:6")

# Payout array
image = [[a1["value"],a2["value"],a3["value"],a3["value"],a5["value"],a6["value"]],
          [b1["value"],b2["value"],b3["value"],b3["value"],b5["value"],b6["value"]],
          [c1["value"],c2["value"],c3["value"],c3["value"],c5["value"],c6["value"]],
          [d1["value"],d2["value"],d3["value"],d3["value"],d5["value"],d6["value"]],
          [e1["value"],e2["value"],e3["value"],e3["value"],e5["value"],e6["value"]],
         ]
pprint(image)


[[15001.6, 15001.6, 2.6, 2.6, 15001.6, 1.6],
 [68250.0, 37502.5, 57017.0, 57017.0, 37502.5, 22501.6],
 [15001.6, 37502.5, 37502.5, 37502.5, 69780.0, 15001.6],
 [40510.0, 37502.5, 37502.5, 37502.5, 37502.5, 2.6],
 [2.6, 15001.6, 15001.6, 15001.6, 41295.0, 15001.6]]


In [181]:
#
# A Turf War Map
#

# Use convenience functions to build value array

a1 = mountains(3000,2000,location="A:1")
a2 = forest(3000,0,2000,location="A:2")
a3 = plains(3000, 2000,location="A:3")
a4 = mountains(1600,1000,location="A:4")
a5 = forest(3000,0,2000,location="A:5")
a6 = plains(3000, 2000,location="A:6")

b1 = factory(1650,500,5000,location="B:1")
b2 = mountains(5000,5000,location="B:2")
b3 = lighthouse(3500,300,7000,location="B:3")
b4 = plains(5000,5000,location="B:4")
b5 = mountains(5000,5000,location="B:5")
b6 = gold_mine(90000,500,5000,location="B:6")

c1 = forest(3000,0,2000,location="C:1")
c2 = plains(5000,5000,location="C:2")
c3 = mountains(5000,5000,location="C:3")
c4 = forest(5000,0,5000,location="C:4")
c5 = hells_gate(600,800,8500,location="C:5")
c6 = mountains(3000,2000,location="C:6")

d1 = mountains(3000,2000,location="D:1")
d2 = witch_hut(3500,600, 7000,location="D:2")
d3 = plains(5000,5000,location="D:3")
d4 = altar(600,800,8500,location="D:4")
d5 = forest(5000,0,5000,location="D:5")
d6 = lighthouse(2000, 400, 5000,location="D:6")

e1 = plains(3000, 2000,location="E:1")
e2 = mountains(3000,2000,location="E:2")
e3 = forest(3000,0,2000,location="E:3")
e4 = plains(3000, 2000,location="E:4")
e5 = mountains(3000,2000,location="E:5")
e6 = forest(3000,0,2000,location="E:6")

# Payout array
image = [[a1["value"],a2["value"],a3["value"],a3["value"],a5["value"],a6["value"]],
          [b1["value"],b2["value"],b3["value"],b3["value"],b5["value"],b6["value"]],
          [c1["value"],c2["value"],c3["value"],c3["value"],c5["value"],c6["value"]],
          [d1["value"],d2["value"],d3["value"],d3["value"],d5["value"],d6["value"]],
          [e1["value"],e2["value"],e3["value"],e3["value"],e5["value"],e6["value"]],
         ]
pprint(image)

[[30003.0, 30003.0, 30003.0, 30003.0, 30003.0, 30003.0],
 [82582.5, 75005.0, 109535.0, 109535.0, 75005.0, 136500.0],
 [30003.0, 75005.0, 75005.0, 75005.0, 139560.0, 30003.0],
 [30003.0, 114035.0, 75005.0, 75005.0, 75005.0, 81020.0],
 [30003.0, 30003.0, 30003.0, 30003.0, 30003.0, 30003.0]]


In [157]:
# NSEW Averaging gives strategic value of a position
# I think of it as an image blur, but iirc it's equivalent to a discrete gradient operator
filter_kernel = [[0, .2, 0],
                 [.2, .2, .2],
                 [0, .2, 0]]

res = convolve2d(image, 
                 filter_kernel,
                 mode='same', 
                 boundary='fill', 
                 fillvalue=0)
pprint(res)

array([[28517.7, 33002.8, 39908.8, 39908.8, 33002.8, 39301.2],
       [43518.7, 74426.1, 79816.6, 79816.6, 98120.6, 54302.2],
       [43518.7, 73810.6, 81911. , 94822. , 78915.6, 77416.6],
       [40808.8, 64810.2, 73810.6, 66004.6, 80118.6, 43206.2],
       [18001.8, 40808.8, 33002.8, 33002.8, 33002.8, 28205.2]])


In [158]:
#
# An approach that shows well-labeled heatmaps.
#

def show_heatmap(arr, title):
  flip = np.flip( np.array(arr), axis=0)  # use numpy to flip array vertically
  z = flip.round()

  x = ['1', '2', '3', '4','5','6']
  y = ['E','D','C','B','A']

  #z_text = ["%.0f" % number for number in z] - doesn't work

  z_text = flip.round( decimals=-1)

  # Possible to build up z_text to make it more friendly
  # z_text = [['Win', 'Lose', 'Win'],
  #          ['Lose', 'Lose', 'Win'],
  #          ['Win', 'Win', 'Lose']]

  fig = ff.create_annotated_heatmap(z, x=x, y=y, annotation_text=z_text, colorscale='Viridis')
  fig.update_layout(title=title, autosize=False,
                  width=500, height=500,
                  margin=dict(l=65, r=50, b=65, t=90))
  fig.show()



In [159]:
show_heatmap(image,"Payout Values for 3rd Rounds")
show_heatmap(res,"Strategic Values for Non-Payout Rounds")

In [160]:
fig = go.Figure(data=[go.Surface(z=image)])
fig.update_layout(title='Payout Value of Positions', autosize=False,
                  width=500, height=500,
                  margin=dict(l=65, r=50, b=65, t=90))
fig.show()

In [161]:
fig = go.Figure(data=[go.Surface(z=res)])
fig.update_layout(title='Strategic Value of Positions', autosize=False,
                  width=500, height=500,
                  margin=dict(l=65, r=50, b=65, t=90))
fig.show()

In [162]:
# Puts data into super-clear dataframe, might use elsewhere.
oneTurn = pd.DataFrame(res, index=["A","B","C","D","E"], columns=[1,2,3,4,5,6])
oneTurn

,1,2,3,4,5,6
A,28517.7,33002.8,39908.8,39908.8,33002.8,39301.2
B,43518.7,74426.1,79816.6,79816.6,98120.6,54302.2
C,43518.7,73810.6,81911.0,94822.0,78915.6,77416.6
D,40808.8,64810.2,73810.6,66004.6,80118.6,43206.2
E,18001.8,40808.8,33002.8,33002.8,33002.8,28205.2


In [163]:
payout = pd.DataFrame(image, index=["A","B","C","D","E"], columns=[1,2,3,4,5,6])
payout

,1,2,3,4,5,6
A,30003.0,30003.0,30003.0,30003.0,30003.0,30003.0
B,82582.5,75005.0,109535.0,109535.0,75005.0,136500.0
C,30003.0,75005.0,75005.0,75005.0,139560.0,30003.0
D,30003.0,114035.0,75005.0,75005.0,75005.0,81020.0
E,30003.0,30003.0,30003.0,30003.0,30003.0,30003.0


In [164]:
row_dict = { 'A': 0, 'B':1, 'C':2, 'D':3, 'E':4}
rows = [ 'A', 'B', 'C', 'D', 'E']
cols = [1,2,3,4,5,6]

def grid_distance(x0,y0, x1,y1):
  return abs(x0-x1) + abs(y0-y1)

#grid_distance(0,0,2,2) == grid_distance(2,2,0,0)

payout.iloc[row_dict['E']][6]

30003.0

In [165]:
for r in rows:
  for c in cols:
    print(f"({r}-{c})",end='')
  print("")

for r in rows:
  for c in cols:
    print( "("+str(payout.iloc[row_dict[r]][c])+")",end='')
  print("")

(A-1)(A-2)(A-3)(A-4)(A-5)(A-6)
(B-1)(B-2)(B-3)(B-4)(B-5)(B-6)
(C-1)(C-2)(C-3)(C-4)(C-5)(C-6)
(D-1)(D-2)(D-3)(D-4)(D-5)(D-6)
(E-1)(E-2)(E-3)(E-4)(E-5)(E-6)
(30003.0)(30003.0)(30003.0)(30003.0)(30003.0)(30003.0)
(82582.5)(75005.0)(109535.0)(109535.0)(75005.0)(136500.0)
(30003.0)(75005.0)(75005.0)(75005.0)(139560.0)(30003.0)
(30003.0)(114035.0)(75005.0)(75005.0)(75005.0)(81020.0)
(30003.0)(30003.0)(30003.0)(30003.0)(30003.0)(30003.0)


In [166]:
#
# Shows the maximum possible single tile you can take
# at a given turn.
#

def oneMove(frame):
  ret = [[0 for x in range(0,6)] for y in range(0,5)] 

  row_num=0
  for r in rows:
    col_num=0
    for c in cols:
      max = 0

      # scan nbr above
      up = chr(ord(r)-1)
      if up in rows:
        tval = frame.iloc[row_dict[up]][c]
        if tval>max:
          max = tval 

      # scan nbr below
      dn = chr(ord(r)+1)
      if dn in rows:
        tval = frame.iloc[row_dict[dn]][c]
        if tval>max:
          max = tval              

      # scan nbr left
      left = c-1
      if left in cols:
        tval = frame.iloc[row_dict[r]][left]
        if tval>max:
          max = tval   

      # scan nbr right
      right = c+1
      if right in cols:
        tval = frame.iloc[row_dict[r]][right]
        if tval>max:
          max = tval   

      # scan my val
      tval = frame.iloc[row_dict[r]][c]
      if tval>max:
         max = tval   

      ret[row_num][col_num] = max
      col_num += 1
      #print(f"({r}-{c})",end='')
    row_num +=1
  return ret

#pprint(ret)
#twoTurn = pd.DataFrame(ret, index=["A","B","C","D","E"], columns=[1,2,3,4,5,6])
#twoTurn

show_heatmap(image,"Strategic Positions for Turns 3n")

foo = oneMove(payout)
show_heatmap(foo,"Strategic Positions for Turns 3n+2")

thisThing = pd.DataFrame(foo, index=["A","B","C","D","E"], columns=[1,2,3,4,5,6])
bar = oneMove(thisThing)
show_heatmap(bar,"Strategic Positions for Turns 3n+1")




In [183]:
#
# Shows payout shares for holding a tile on round 3n 
#

def append_value(ret, value, label):
  if value>0:
    if ret is not "":
      ret = ret + ", "
    ret = ret +  f"{value:,.0f} {label}"
  return ret

def share(tile, percent):
  ret = ""
  ret = append_value(ret,tile[Currency.COMMON_SOUL]  * percent, "common souls" )
  ret = append_value(ret,tile[Currency.RARE_SOUL] * percent, "rare souls" )
  ret = append_value(ret,tile[Currency.SOUL_DUST] * percent, "soul dust" )
  ret = append_value(ret,tile[Currency.EPIC_SOUL] * percent, "epic souls" )
  ret = append_value(ret,tile[Currency.GOLD] * percent, "gold" )
  ret = append_value(ret,tile[Currency.LEGENDARY_SOUL] * percent, "legendary souls" )
  ret = append_value(ret,tile[Currency.CONTRIBUTION] * percent, "contribution" )
  #ret = append_value(ret,tile[Currency.INFLUENCE] * percent, "gold" )
  ret = append_value(ret,tile[Currency.SPARK] * percent, "sparks" )
  ret = append_value(ret,tile[Currency.GEM] * percent, "gems" )
  return ret

gm = b6 # c5 # gold_mine(90000,500,5000)

print("```")
print("Payouts for " + gm["location"] + " " + gm["name"])
print("Place               Reward")
print("First               " + share(gm,.07))
print("2nd                 " + share(gm,.06))
print("3rd                 " + share(gm,.05))
print("4-6                 " + share(gm,.04))
print("7-11                " + share(gm,.03))
print("12-25               " + share(gm,.02))
print("26-44               " + share(gm,.01))
print("```")


```
Payouts for B:6 Gold Mine
Place               Reward
First               6,300 gold, 350 contribution
2nd                 5,400 gold, 300 contribution
3rd                 4,500 gold, 250 contribution
4-6                 3,600 gold, 200 contribution
7-11                2,700 gold, 150 contribution
12-25               1,800 gold, 100 contribution
26-44               900 gold, 50 contribution
```


# Notes


1.   Annotated heatmap: https://plotly.com/python/annotated-heatmap/
2.   A-Star search on grid in python at https://github.com/BaijayantaRoy/Medium-Article/blob/master/A_Star.ipynb

